# Seminar: Computational Mathematics
$\newcommand{\vect}[1]{\boldsymbol{#1}}$

In [115]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

## Geometry and Mesh

In [116]:
square = MoveTo(0,0).Rectangle(5,1).Face()
hole = Circle((2.5,0.5), 0.2).Face()
hole.edges.name = "circle"
square.edges.Max(Y).name = "top"
square.edges.Min(Y).name = "bottom"
square.edges.Min(X).name = "inlet"
square.edges.Max(X).name = "outlet"
shape = square - hole

In [117]:
geo = OCCGeometry(shape, dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=0.1)).Curve(3)
#Draw(mesh)

## Deformation (Dirichlet)

In [118]:
fes_def = VectorH1(mesh, order=3, dirichlet=".*")

We create a smooth deformation by solving a Poisson equation in discrete form $ A \ u = f $ with $ f= 0 $.

The bilinear form $A$ includes both the FreeDofs and the fixed Dirichlet DOFs:
$$ A \ u = \begin{pmatrix} A_f & A_D \end{pmatrix} \begin{pmatrix} u_f \\ u_D \end{pmatrix} = 0 $$

In [119]:
u_def, v_def = fes_def.TnT()
a_def = BilinearForm(InnerProduct(grad(u_def), grad(v_def))*dx).Assemble()
A_def_inv = a_def.mat.Inverse(freedofs=fes_def.FreeDofs())

To get $u$, we first set a fixed deformation at the boundary, `gf_def`, corresponding to $u_D$.

In [120]:
gf_def = GridFunction(fes_def) # this will hold all deformations
gf_def_top = GridFunction(fes_def)
gf_def_top.Set((0, 2*x*(5-x)), definedon=mesh.Boundaries("top"))

From this we obtain $f_D$, using the `*`-operator (matrix-vector product):
$ f_D = -A \begin{pmatrix} 0 \\ u_D \end{pmatrix} $ \
Finally, we solve for $u_f$ by using only the `FreeDofs` $$ u_f = A_f^{-1} \cdot f_D $$ with the `@`-operator, and add this to the total deformation.

In [121]:
gf_def_top.vec.data -= A_def_inv@a_def.mat*gf_def_top.vec      # Matrix @ Matrix * Vector
#gf_def_top.vec.data -= A_def_inv*(a_def.mat*gf_def_top.vec)   # Matrix * Vector

In [122]:
# initial values for deformations
delta_top = Parameter(-0.005)

# combine all deformations into one grid function
gf_def.vec.data = delta_top.Get() * gf_def_top.vec  # + ...

In [123]:
#Draw(gf_def, deformation=gf_def)

## Deformation (Bending)

With the previous method we had to specify the full displacement vector $u$ on the Dirichlet boundary. Now we want to set only one component ($u_y$) on one of the sides (outlet) and let the displacement on the rest of the boundary be calculated automatically by, again, solving a Poisson problem.

To achieve this, we can use a mixed formulation, applying Dirichlet boundary conditions only on one component, while the remaining components will have zero Neumann BC. Test functions $v$ are not required to be zero on the new Dirichlet boundary, which adds a boundary term to the equation:
$$ \int \nabla u \nabla v - \int_{\Gamma_D} \frac{\partial u_y}{\partial n} v_y = \int f v $$
We introduce a new variable $$\lambda = -\frac{\partial u_y}{\partial n}$$
and the mixed problem ($f=0$):

Find $u \in V := [H^1(\Omega)]^d$ and $\lambda \in Q$ such that
$$
\begin{array}{ccccll}
\int_\Omega \nabla u \nabla v & + & \int_{\Gamma_D} \lambda \ v_y & = & 0 & \forall \, v \in V \\
\int_{\Gamma_D} \mu \ u_y & & & = & \int_{\Gamma_D} \mu \ u_D & \forall \, \mu \in Q
\end{array}
$$

For $u_D=1$ it is enough to choose a 1-dimensional space: $Q = \mathbb{R}$.
In general one would use $Q := H^{-1/2}(\Gamma_D)$ or $L^2$.

However, since the Poisson problem is still decoupled for the different coordinates, there is still no displacement in x-direction. To get a more realistic deformation, we symmetrize the gradients:
$$ \text{Sym}(\nabla u) = \frac{1}{2} (\nabla u + \nabla u^T) =: \varepsilon(u) $$
This is like the mechanical strain tensor $\varepsilon$, and we get $\varepsilon(u) : \varepsilon(v)$ in the first integral.

In [124]:
mesh.SetDeformation(None)

fes_def_2 = VectorH1(mesh, order=3, dirichlet="inlet")
fes_number = NumberSpace(mesh)
X = fes_def_2 * fes_number

(u_def_2, lam), (v_def_2, mu) = X.TnT()

In [125]:
a_def_2 = BilinearForm(X)
a_def_2 += InnerProduct(Sym(grad(u_def_2)), Sym(grad(v_def_2)))*dx
# Same as:
#a_def_2 += InnerProduct(0.5*(grad(u_def_2) + grad(u_def_2).trans), 0.5*(grad(v_def_2) + grad(v_def_2).trans))*dx
a_def_2 += (u_def_2[1]*mu + v_def_2[1]*lam)*ds("outlet")
a_def_2.Assemble()

A_def_inv_2 = a_def_2.mat.Inverse(freedofs=X.FreeDofs())

In [126]:
gf_mean_out = GridFunction(X)
l_def_2 = LinearForm(X)
l_def_2 += 1*mu*ds("outlet")
l_def_2.Assemble()

gf_mean_out.vec.data = A_def_inv_2*l_def_2.vec

In [127]:
delta_out = Parameter(0.3)

# add deformation to "global" deformation function
gf_def.vec.data += delta_out.Get()*gf_mean_out.components[0].vec

In [128]:
#mesh.SetDeformation(gf_def)

In [129]:
#Draw(gf_mean_out.components[0])
Draw(gf_def, deformation=gf_def)
#Draw(gf_def.components[1], deformation=gf_def)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

# Piola Transformation
(Book Ch. 8.5.1)

Simple coordinate transformations don't preserve the divergence of the fluid flow.

Piola transformation $\tilde{\vect{u}}_P = \mathbf{P}_\vect{\Phi}(u)$, $\quad \tilde{\vect{x}} = \vect{\Phi}(\vect{x})$:
$$ \tilde{\vect{u}}_P(\tilde{\vect{x}}) = \frac{1}{\left|J_\vect{\Phi}(\vect{x}) \right| } J_\vect{\Phi}(\vect{x}) \vect{u}(\vect{x}) $$

$\DeclareMathOperator{\opdiv}{div}$
$$ H(\opdiv) = \{ \sigma \in [L_2]^2 : \opdiv \sigma \in L_2 \} $$

$$ \int_\Omega \opdiv \sigma \varphi = -\int_\Omega \sigma \nabla \varphi + \int_{\partial \Omega} \sigma_n \varphi $$

# Reduced Basis Method

Parameter vector $\vect{\mu}= (\mu_1, ..., \mu_P)^T $ with geometric parameters $\vect{\mu}_g$ and physical parameters $\vect{\mu}_{ph}$.

Take snapshots of high-fidelity FEM model and use them as basis vectors for a reduced-order model.

### Stokes Equations
Flow of a Newtonian incompressible viscous fluid, when convective forces are negligible \
(Book Ch. 2.1.3)

Velocity $\vect{u}$, pressure $p$, kinematic viscosity $\nu$ (constant). \
Domain $\Omega \subset \mathbb{R}^d$, boundary $\Gamma$.
\begin{align}  -\nu \ \Delta \vect{u} + \nabla p &= \vect{0}  && \text{in} \ \Omega \\
  \text{div} \ \vect{u} &= 0  && \text{in} \ \Omega \\
  \vect{u} &= \vect{g}  && \text{on} \ \Gamma_D \\
  -p \vect{n} + \nu (\nabla \vect{u}) \vect{n} &= \vect{h}  && \text{on} \ \Gamma_N  \end{align}
First equation: Conservation of linear momentum \
Second equation: Conservation of mass (Incompressibility condition)

### Navier-Stokes Equations
Stokes with added convective term in the momentum equation \
(Book Ch. 2.1.4)

\begin{align}  -\nu \ \Delta \vect{u} + (\vect{u} \cdot \nabla) \vect{u} + \nabla p &= \vect{0}  && \text{in} \ \Omega \\
  \text{div} \ \vect{u} &= 0  && \text{in} \ \Omega \\
  \vect{u} &= \vect{g}  && \text{on} \ \Gamma_D \\
  -p \vect{n} + \nu (\nabla \vect{u}) \vect{n} &= \vect{h}  && \text{on} \ \Gamma_N  \end{align}

### Stokes - Weak Form

$$ -\nu \ v_i \Delta u_i + v_i \frac{\partial p}{\partial x_i} = 0 $$
$$ \nu \ \nabla v_i \nabla u_i - \frac{\partial v_i}{\partial x_i} p = 0 $$
$$ \nu \ \nabla \vect{v} \nabla \vect{u} + ... $$

$\newcommand{\TEST}[1]{\boldsymbol{#1}}$
$\TEST{v}$